In [1]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time
import pandas

import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection, Point, MultiPoint
from shapely.affinity import translate

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.CATCH_utils import *

In [2]:
# basepaths
bp       = '/Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL'
bp_ws    = '/Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL/workspaces'
bp_annos = '/Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL/Gold'
bp_cases = '/Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL/Cases'
bp_imgs  = '/Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL/Imgs'

In [4]:
# unpack the workspaces from ws_path to case_storage_path
#cvi42ws_paths     = [f for f in os.listdir(bp_ws) if f.endswith('cvi42ws')]
#parse_cvi42ws(bp_ws, bp_annos, process=True, debug=True)

#parse_cvi42ws(bp_ws, bp_annos, process=True, debug=False)


# get the paths of imgs and annotations via SeriesInstanceUIDs
imgsanno_paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)

cases = []
sax_cine_view = SAX_CINE_View()
#sax_cs_view   = SAX_CS_View()
for count, (imgp,annop) in enumerate(imgsanno_paths):
    if count < 2: continue
    print(imgp, annop)
    if not os.path.exists(imgp) or not os.path.exists(annop):
        print(os.path.exists(imgp), os.path.exists(annop))
        continue
    st = time()
    #try:
    case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
    case = sax_cine_view.customize_case(case)
    #case = sax_cs_view.customize_case(case)
    case.store(bp_cases)
    cases.append(case)
    print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)
    #except Exception as e:
    #    print('Case customization failed: ', e)
    

/Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL/Imgs/CAMAID_003_1 /Users/dietrichhadler/Desktop/Daten/CAMAID_new_LL/Gold/1.3.12.2.1107.5.2.41.69541.30000021043005072317400000006
nr images here:  480


AttributeError: 'FileDataset' object has no attribute 'SliceLocation'

In [ ]:
all_cases = os.listdir(bp_imgs)
print('All cases: ', all_cases)

row_dict = {}
row_counter = 0
columns = ['case', 'sd', 'suid']
for i_c, c in enumerate(all_cases):
    for ip, p in enumerate(Path(os.path.join(bp_imgs, c)).glob('**/*.dcm')):
        p = str(p)
        dcm = pydicom.dcmread(p, stop_before_pixels=False)
        #match = CS_SAX.match(dcm.SeriesDescription)
        #print(dcm.SeriesDescription, bool(match))
        #if not match: add_LL_tag(p, dcm, tag='Lazy Luna: None')
        #else:         add_LL_tag(p, dcm, tag='Lazy Luna: SAX CS')
        try:
            row = [c, dcm.SeriesDescription, dcm.SeriesInstanceUID]
            row_dict['row_'+str(row_counter).zfill(6)] = row
            row_counter += 1
        except: continue
    print(i_c, 'Done for', c)
    #if i_c==1: break

df = pandas.DataFrame.from_dict(row_dict, orient='index', columns=columns)
display(df)

In [ ]:
print('Cine SAX images:')
for k,v in sorted(df[['case','sd']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    if 'RV 7 0' in k[1]:
        print('\t', k[1], v)

In [ ]:
print('Cine SAX images:')
for k,v in sorted(df[['case','sd']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    if 'RV 7 0' in k[1]:
        print('\t', k[1], v)
print('CASE 003 1 is OFF BY 1')

In [ ]:
print('4CH View images:')

curr = None
for k,v in sorted(df[['case','sd']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    if 'cine_tf2d12' in k[1] and ('2 cv' in k[1] or '2cv' in k[1]):
        print('\t', k[1],v)
    if 'cine_tf2d12' in k[1] and ('3 cv' in k[1] or '3cv' in k[1]):
        print('\t', k[1], v)
    if 'cine_tf2d12' in k[1] and ('4 cv' in k[1] or '4cv' in k[1]):
        print('\t', k[1], v)
print('Several cases have two series of 4cv')

In [ ]:
print('T1 images:')
for k,v in sorted(df[['case','sd']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    #if curr!='Camaid_011_2': continue
    if 'pre_MOLLI' in k[1] and 'T1' in k[1] and 'T1S' not in k[1] and '4cv' not in k[1]:
        print('\t', k[1], v)
    #print('\t', k[1], v)
print('T1 fehlt bei Camaid 11')

In [ ]:
print('T2 images:')
for k,v in sorted(df[['case','sd']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    #if curr!='Camaid_011_2': continue
    if 'T2' in k[1] and 'MOCO_T2' in k[1] and '4cv' not in k[1]:
        print('\t', k[1], v)
    #print('\t', k[1], v)
print('Viele Fälle bereits 3 aus 5 definiert.')

In [ ]:
print('LGE images:')
for k,v in sorted(df[['case','sd']].value_counts().iteritems()):
    if curr is None or curr!=k[0]: print(k[0])
    curr = k[0]
    #if curr!='Camaid_011_2': continue
    if 'LGE' in k[1]:
        print('\t', k[1], v)
    #print('\t', k[1], v)
print('')

In [ ]:
def add_LL_tag(store_path, dcm, tag='Lazy Luna: None'): # Lazy Luna: SAX CS
    try:    dcm[0x0b, 0x10].value = tag
    except: dcm.private_block(0x000b, tag, create=True)
    dcm.save_as(filename=store_path, write_like_original=False)
    
all_cases = os.listdir(bp_imgs)
print('All cases: ', all_cases)

columns = ['case', 'sd', 'suid']
for i_c, c in enumerate(all_cases):
    for ip, p in enumerate(Path(os.path.join(bp_imgs, c)).glob('**/*.dcm')):
        p = str(p)
        dcm = pydicom.dcmread(p, stop_before_pixels=False)
        #print(dcm.SeriesDescription)
        try:
            if 'RV 7 0' in dcm.SeriesDescription: add_LL_tag(p, dcm, tag='Lazy Luna: SAX CINE')
            else:                                 add_LL_tag(p, dcm, tag='Lazy Luna: None')
        except: 
            continue
    print(i_c, 'Done for', c)


In [ ]:
for i_c, c in enumerate(all_cases):
    for ip, p in enumerate(Path(os.path.join(bp_imgs, c)).glob('**/*.dcm')):
        p = str(p)
        dcm = pydicom.dcmread(p, stop_before_pixels=False)
        name = str(dcm[0x0b, 0x10].value).replace('Lazy Luna: ', '') # LL Tag
        print(name)